### GALAXY PAIRS CATALOG

In [5]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
from tqdm.notebook import tqdm
from helper import preprocess_catalog_galactic
from helper import load_catalog

In [6]:
# --- Settings ---
dataset = "BOSS"
if dataset == "eBOSS":
    catalog, region = "LRG", "NGC"
    real_file = f"data/eBOSS/eBOSS_{catalog}_clustering_data-{region}-vDR16.fits"
    rand_file = f"data/eBOSS/eBOSS_LRG_clustering_random-{region}-vDR16.fits"
elif dataset == "BOSS":
    catalog, region = "CMASS", "North"
    real_file = f"data/BOSS/galaxy_DR12v5_CMASS_{region}.fits"
    rand_file = f"data/BOSS/random0_DR12v5_CMASS_{region}.fits"
else:
    raise ValueError("dataset must be eBOSS or BOSS")

alm_file = "data/COM_Lensing_4096_R3.00/MV/dat_klm.fits"
mask_file = "data/COM_Lensing_4096_R3.00/mask.fits"

In [7]:
def compute_angle_cosine(l1, b1, l2, b2):
    """
    Compute cosine of angle θ between two directions (in degrees) in Galactic coordinates.
    """
    l1_rad, b1_rad = np.radians(l1), np.radians(b1)
    l2_rad, b2_rad = np.radians(l2), np.radians(b2)

    cos_theta = (
        np.cos(b1_rad) * np.cos(l1_rad) * np.cos(b2_rad) * np.cos(l2_rad) +
        np.cos(b1_rad) * np.sin(l1_rad) * np.cos(b2_rad) * np.sin(l2_rad) +
        np.sin(b1_rad) * np.sin(b2_rad)
    )
    return cos_theta

def build_galaxy_pair_catalog(data, weights, r_par_max=20, r_perp_min=18, r_perp_max=22):
    """
    Build galaxy pair catalog based on parallel and perpendicular distance criteria.
    Returns a list of dictionaries.
    """
    l, b, D, data_filtered, weights_valid = preprocess_catalog_galactic(data, weights)
    # weights = weights[:len(data_filtered)]
    z = data_filtered['Z']

    pairs = []

    for i in tqdm(range(len(data_filtered)), desc="Building Pair Catalog"):
        for j in range(i + 1, len(data_filtered)):
            Dc1, Dc2 = D[i], D[j]
            r_par = np.abs(Dc2 - Dc1)  # cos(θ) ≈ 1

            if r_par > r_par_max:
                continue

            cos_theta = compute_angle_cosine(l[i], b[i], l[j], b[j])
            theta = np.arccos(np.clip(cos_theta, -1, 1))

            r_perp = ((Dc2 + Dc1) / 2.0) * theta  # correct small-angle formula in Mpc/h

            if r_perp_min <= r_perp <= r_perp_max:
                Dmid = cosmo.comoving_distance((z[i] + z[j]) / 2).value * cosmo.h  # Mpc/h
                pairs.append({
                    'l1': l[i], 'b1': b[i], 'z1': z[i], 'w1': weights_valid[i], 'Dc1': Dc1, 'ID1': data_filtered[i]['ID'],
                    'l2': l[j], 'b2': b[j], 'z2': z[j], 'w2': weights_valid[j], 'Dc2': Dc2, 'ID2': data_filtered[j]['ID'],
                    'Dmid': Dmid
                })

    pairs = pd.DataFrame(pairs)
    pairs.to_csv(f"data/paircatalogs/galaxy_pairs_catalog_{catalog}_{region}_{r_par_max}_{r_perp_min}_{r_perp_max}hmpc.csv", index=False)
    return pairs

In [ ]:
# --- Run all ---
if dataset == "BOSS":
    z_min = .4
    z_max = .7
    weight = "CMASS"
else: # eBOSS
    z_min = 0
    z_max = 10000
    weight = True

data_real, w_real = load_catalog(real_file, weights=weight, z_min=z_min, z_max=z_max)
# data_rand, w_rand = load_catalog(rand_file, random_fraction=0.10)

# Build galaxy pair catalog
pair_catalog = build_galaxy_pair_catalog(data_real, w_real, r_par_max=30, r_perp_min=0, r_perp_max=30)
print(f"Total valid pairs: {len(pair_catalog)}")

# Jackknife real galaxies
# kappa_real, sigma_real = jackknife_stack_healpix(data_real, w_real, data_rand, nside=10)
# sn_real = np.zeros_like(kappa_real)
# valid = sigma_real > 0
# sn_real[valid] = kappa_real[valid] / sigma_real[valid]
# kappa_rand, sigma_rand, sn_rand = stack_kappa(data_rand, w_rand, "Random")
# kappa_sub = kappa_real - kappa_rand
# kappa_smooth = gaussian_filter(kappa_sub, sigma=2)

Building Pair Catalog:   0%|          | 0/579089 [00:00<?, ?it/s]